In [1]:
pip install graphviz

In [2]:
# Importaciones necesarias
from typing import Optional, Any, Tuple, List
from graphviz import Digraph
import csv

class Nodo:
    """
    Representa un nodo en el árbol AVL.
    Cada nodo contiene información sobre una película y sus ganancias.
    """
    def __init__(self, titulo: str, ganancias_mundiales: float, ganancias_domesticas: float,
                 porcentaje_ganancias_domesticas: float, ganancias_extranjeras: float,
                 porcentaje_ganancias_extranjeras: float, anio: int, izquierda=None, derecha=None, altura=1):
        self.titulo = titulo
        self.ganancias_mundiales = ganancias_mundiales
        self.ganancias_domesticas = ganancias_domesticas
        self.porcentaje_ganancias_domesticas = porcentaje_ganancias_domesticas
        self.ganancias_extranjeras = ganancias_extranjeras
        self.porcentaje_ganancias_extranjeras = porcentaje_ganancias_extranjeras
        self.anio = anio
        self.izquierda = izquierda
        self.derecha = derecha
        self.altura = altura

    def __str__(self):
        return f"Título: {self.titulo}, Año: {self.anio}"

class ArbolAVL:
    """
    Implementa un árbol AVL para almacenar y gestionar datos de películas.
    """
    def __init__(self):
        self.raiz = None

    def altura(self, nodo: Optional[Nodo]) -> int:
        """Retorna la altura de un nodo dado."""
        return nodo.altura if nodo else 0

    def actualizar_altura(self, nodo: Nodo) -> None:
        """Actualiza la altura de un nodo basándose en las alturas de sus hijos."""
        nodo.altura = 1 + max(self.altura(nodo.izquierda), self.altura(nodo.derecha))

    def factor_equilibrio(self, nodo: Optional[Nodo]) -> int:
        """Calcula el factor de equilibrio de un nodo."""
        return self.altura(nodo.derecha) - self.altura(nodo.izquierda) if nodo else 0

    def rotar_izquierda(self, z: Nodo) -> Nodo:
        """Realiza una rotación izquierda en el árbol AVL."""
        y = z.derecha
        T2 = y.izquierda

        y.izquierda = z
        z.derecha = T2

        self.actualizar_altura(z)
        self.actualizar_altura(y)

        return y

    def rotar_derecha(self, y: Nodo) -> Nodo:
        """Realiza una rotación derecha en el árbol AVL."""
        x = y.izquierda
        T2 = x.derecha

        x.derecha = y
        y.izquierda = T2

        self.actualizar_altura(y)
        self.actualizar_altura(x)

        return x

    def insertar(self, nodo: Nodo) -> None:
        """Inserta un nuevo nodo en el árbol AVL."""
        self.raiz = self._insertar_recursivo(self.raiz, nodo)

    def _insertar_recursivo(self, raiz: Optional[Nodo], nodo: Nodo) -> Nodo:
        """Función auxiliar recursiva para insertar un nodo y mantener el equilibrio del árbol."""
        if not raiz:
            return nodo

        if nodo.titulo < raiz.titulo:
            raiz.izquierda = self._insertar_recursivo(raiz.izquierda, nodo)
        else:
            raiz.derecha = self._insertar_recursivo(raiz.derecha, nodo)

        self.actualizar_altura(raiz)

        equilibrio = self.factor_equilibrio(raiz)

        # Casos de desequilibrio y reequilibrio
        if equilibrio < -1 and nodo.titulo < raiz.izquierda.titulo:
            return self.rotar_derecha(raiz)

        if equilibrio > 1 and nodo.titulo > raiz.derecha.titulo:
            return self.rotar_izquierda(raiz)

        if equilibrio < -1 and nodo.titulo > raiz.izquierda.titulo:
            raiz.izquierda = self.rotar_izquierda(raiz.izquierda)
            return self.rotar_derecha(raiz)

        if equilibrio > 1 and nodo.titulo < raiz.derecha.titulo:
            raiz.derecha = self.rotar_derecha(raiz.derecha)
            return self.rotar_izquierda(raiz)

        return raiz

    def eliminar(self, titulo: str) -> None:
        """Elimina un nodo del árbol AVL basado en el título de la película."""
        self.raiz = self._eliminar_recursivo(self.raiz, titulo)

    def _eliminar_recursivo(self, raiz: Optional[Nodo], titulo: str) -> Optional[Nodo]:
        """Función auxiliar recursiva para eliminar un nodo y mantener el equilibrio del árbol."""
        if not raiz:
            return raiz

        if titulo < raiz.titulo:
            raiz.izquierda = self._eliminar_recursivo(raiz.izquierda, titulo)
        elif titulo > raiz.titulo:
            raiz.derecha = self._eliminar_recursivo(raiz.derecha, titulo)
        else:
            if not raiz.izquierda:
                return raiz.derecha
            elif not raiz.derecha:
                return raiz.izquierda

            temp = self._encontrar_minimo(raiz.derecha)
            raiz.titulo = temp.titulo
            raiz.derecha = self._eliminar_recursivo(raiz.derecha, temp.titulo)

        if not raiz:
            return raiz

        self.actualizar_altura(raiz)

        equilibrio = self.factor_equilibrio(raiz)

        # Casos de desequilibrio y reequilibrio después de la eliminación
        if equilibrio < -1 and self.factor_equilibrio(raiz.izquierda) <= 0:
            return self.rotar_derecha(raiz)

        if equilibrio < -1 and self.factor_equilibrio(raiz.izquierda) > 0:
            raiz.izquierda = self.rotar_izquierda(raiz.izquierda)
            return self.rotar_derecha(raiz)

        if equilibrio > 1 and self.factor_equilibrio(raiz.derecha) >= 0:
            return self.rotar_izquierda(raiz)

        if equilibrio > 1 and self.factor_equilibrio(raiz.derecha) < 0:
            raiz.derecha = self.rotar_derecha(raiz.derecha)
            return self.rotar_izquierda(raiz)

        return raiz

    def _encontrar_minimo(self, nodo: Nodo) -> Nodo:
        """Encuentra el nodo con el valor mínimo en un subárbol."""
        actual = nodo
        while actual.izquierda:
            actual = actual.izquierda
        return actual

    def encontrar_padre(self, titulo: str) -> Optional[Nodo]:
        """Encuentra el nodo padre de un nodo dado por su título."""
        if not self.raiz or self.raiz.titulo == titulo:
            return None
        return self._encontrar_padre_recursivo(self.raiz, titulo)

    def _encontrar_padre_recursivo(self, nodo: Nodo, titulo: str) -> Optional[Nodo]:
        """Función auxiliar recursiva para encontrar el nodo padre."""
        if (nodo.izquierda and nodo.izquierda.titulo == titulo) or (nodo.derecha and nodo.derecha.titulo == titulo):
            return nodo
        if titulo < nodo.titulo and nodo.izquierda:
            return self._encontrar_padre_recursivo(nodo.izquierda, titulo)
        if titulo > nodo.titulo and nodo.derecha:
            return self._encontrar_padre_recursivo(nodo.derecha, titulo)
        return None

    def encontrar_tio(self, titulo: str) -> Optional[Nodo]:
        """Encuentra el nodo tío de un nodo dado por su título."""
        padre = self.encontrar_padre(titulo)
        if not padre:
            return None
        abuelo = self.encontrar_padre(padre.titulo)
        if not abuelo:
            return None
        if abuelo.izquierda == padre:
            return abuelo.derecha
        else:
            return abuelo.izquierda

    def encontrar_abuelo(self, titulo: str) -> Optional[Nodo]:
        """Encuentra el nodo abuelo de un nodo dado por su título."""
        padre = self.encontrar_padre(titulo)
        if not padre:
            return None
        return self.encontrar_padre(padre.titulo)

    def buscar(self, titulo: str) -> Optional[Nodo]:
        """Busca un nodo en el árbol por su título."""
        return self._buscar_recursivo(self.raiz, titulo)

    def _buscar_recursivo(self, raiz: Optional[Nodo], titulo: str) -> Optional[Nodo]:
        """Función auxiliar recursiva para buscar un nodo por su título."""
        if not raiz or raiz.titulo == titulo:
            return raiz

        if titulo < raiz.titulo:
            return self._buscar_recursivo(raiz.izquierda, titulo)
        return self._buscar_recursivo(raiz.derecha, titulo)

    def recorrido_inorden(self) -> None:
        """Realiza un recorrido inorden del árbol e imprime los nodos."""
        self._inorden_recursivo(self.raiz)
        print()

    def _inorden_recursivo(self, nodo: Optional[Nodo]) -> None:
        """Función auxiliar recursiva para el recorrido inorden."""
        if nodo:
            self._inorden_recursivo(nodo.izquierda)
            print(nodo, end=" | ")
            self._inorden_recursivo(nodo.derecha)

    def buscar_por_atributos(self, anio: int, valor: float) -> None:
        """Busca nodos que cumplan ciertos criterios de año y ganancias."""
        self._buscar_por_atributos_recursivo(self.raiz, anio, valor)

    def _buscar_por_atributos_recursivo(self, nodo: Optional[Nodo], anio: int, valor: float) -> None:
        """Función auxiliar recursiva para buscar nodos por atributos específicos."""
        if nodo:
            if (nodo.anio == anio and
                nodo.porcentaje_ganancias_domesticas < nodo.porcentaje_ganancias_extranjeras and
                nodo.ganancias_extranjeras > valor):
                print("-----------------------------------")
                print(nodo)
                print(f"El nivel del nodo es: {self.obtener_nivel_nodo(nodo.titulo)}")
                print(f"El factor de equilibrio es: {self.factor_equilibrio(nodo)}")
                padre = self.encontrar_padre(nodo.titulo)
                print(f"El padre del Nodo es: {padre.titulo if padre else 'Ninguno'}")
                abuelo = self.encontrar_abuelo(nodo.titulo)
                print(f"El Abuelo del Nodo es: {abuelo.titulo if abuelo else 'Ninguno'}")
                tio = self.encontrar_tio(nodo.titulo)
                print(f"El Tío del Nodo es: {tio.titulo if tio else 'Ninguno'}")
            self._buscar_por_atributos_recursivo(nodo.izquierda, anio, valor)
            self._buscar_por_atributos_recursivo(nodo.derecha, anio, valor)

    def cargar_desde_csv(self, ruta_archivo: str, limite: int = 20) -> None:
        """Carga datos desde un archivo CSV y los inserta en el árbol AVL."""
        try:
            with open(ruta_archivo, 'r', newline='', encoding='utf-8') as archivo_csv:
                lector_csv = csv.reader(archivo_csv)
                next(lector_csv)  # Saltar la fila de encabezados
                for i, fila in enumerate(lector_csv):
                    if i >= limite:
                        break
                    try:
                        titulo = fila[0]
                        ganancias_mundiales = float(fila[1])
                        ganancias_domesticas = float(fila[2])
                        porcentaje_ganancias_domesticas = float(fila[3])
                        ganancias_extranjeras = float(fila[4])
                        porcentaje_ganancias_extranjeras = float(fila[5])
                        anio = int(fila[6])

                        nodo = Nodo(titulo, ganancias_mundiales, ganancias_domesticas,
                                    porcentaje_ganancias_domesticas, ganancias_extranjeras,
                                    porcentaje_ganancias_extranjeras, anio)
                        self.insertar(nodo)
                    except (ValueError, IndexError) as e:
                        print(f"Error al procesar la fila {i + 2}: {e}")
                        continue
        except FileNotFoundError:
            print(f"Archivo no encontrado: {ruta_archivo}")
        except Exception as e:
            print(f"Ocurrió un error al leer el archivo CSV: {e}")

    def recorrido_por_niveles(self) -> List[List[Nodo]]:
        """Realiza un recorrido por niveles del árbol AVL."""
        if not self.raiz:
            return []

        resultado = []
        self._recorrido_por_niveles_recursivo([self.raiz], resultado)
        return resultado

    def obtener_nivel_nodo(self, titulo: str) -> Optional[int]:
        """Obtiene el nivel de un nodo en el árbol dado su título."""
        return self._obtener_nivel_nodo_recursivo(self.raiz, titulo, 1) - 1

    def _obtener_nivel_nodo_recursivo(self, nodo: Optional[Nodo], titulo: str, nivel: int) -> Optional[int]:
        """Función auxiliar recursiva para obtener el nivel de un nodo."""
        if not nodo:
            return None

        if nodo.titulo == titulo:
            return nivel

        resultado_izquierda = self._obtener_nivel_nodo_recursivo(nodo.izquierda, titulo, nivel + 1)
        if resultado_izquierda:
            return resultado_izquierda

        return self._obtener_nivel_nodo_recursivo(nodo.derecha, titulo, nivel + 1)

    def visualizar(self, nombre_archivo: str = "arbol_avl") -> None:
        """
        Genera una visualización gráfica del árbol AVL utilizando Graphviz.

        :param nombre_archivo: Nombre del archivo de salida para la visualización.
        """
        dot = Digraph(comment='Árbol AVL')
        dot.attr(rankdir='TB')

        def agregar_nodos_aristas(nodo: Optional[Nodo]):
            """Función auxiliar para agregar nodos y aristas al gráfico."""
            if nodo:
                dot.node(str(id(nodo)), f"{nodo.titulo}\n{nodo.anio}")
                if nodo.izquierda:
                    dot.edge(str(id(nodo)), str(id(nodo.izquierda)))
                    agregar_nodos_aristas(nodo.izquierda)
                if nodo.derecha:
                    dot.edge(str(id(nodo)), str(id(nodo.derecha)))
                    agregar_nodos_aristas(nodo.derecha)

        agregar_nodos_aristas(self.raiz)
        dot.render(nombre_archivo, view=True, format='png')

    def _recorrido_por_niveles_recursivo(self, nivel_actual: List[Nodo], resultado: List[List[Nodo]]) -> None:
        """Función auxiliar recursiva para el recorrido por niveles."""
        if not nivel_actual:
            return

        resultado.append(nivel_actual)
        siguiente_nivel = []
        for nodo in nivel_actual:
            if nodo.izquierda:
                siguiente_nivel.append(nodo.izquierda)
            if nodo.derecha:
                siguiente_nivel.append(nodo.derecha)

        self._recorrido_por_niveles_recursivo(siguiente_nivel, resultado)

In [4]:
from typing import Optional
from graphviz import Digraph
import csv

class SistemaGestionPeliculasAVL:
    """
    Sistema de gestión de películas utilizando un Árbol AVL.
    Esta clase proporciona una interfaz para interactuar con el árbol AVL de películas.
    """
    def __init__(self):
        self.arbol_avl = ArbolAVL()
        self.contador_visualizaciones = 0

    def ejecutar(self):
        """
        Método principal que inicia el sistema y maneja el menú de opciones.
        """
        # Cargar datos desde el archivo CSV
        ruta_archivo_csv = "/content/dataset_movies(1).csv"
        self.arbol_avl.cargar_desde_csv(ruta_archivo_csv)

        # Visualizar el árbol inicial
        self.visualizar_arbol("arbol_inicial")
        print("Árbol AVL de películas creado desde CSV y visualizado.")

        opcion = 0
        while opcion != 6:
            self.mostrar_menu()
            try:
                opcion = int(input("Opción seleccionada: "))
                self.procesar_opcion(opcion)
            except ValueError:
                print("Error: Entrada inválida. Por favor, ingrese un número.")

    def mostrar_menu(self):
        """
        Muestra el menú de opciones disponibles para el usuario.
        """
        print("""
        Digite el número de la opción que desea aplicar:
        1. Insertar Nodo.
        2. Eliminar Nodo.
        3. Buscar Nodo a partir del nombre.
        4. Buscar los nodos que cumplan con los criterios establecidos.
        5. Mostrar el recorrido por niveles.
        6. Salir.
        """)

    def procesar_opcion(self, opcion):
        """
        Procesa la opción seleccionada por el usuario.

        :param opcion: Número de la opción seleccionada.
        """
        if opcion == 1:
            self.insertar_nodo()
        elif opcion == 2:
            self.eliminar_nodo()
        elif opcion == 3:
            self.buscar_nodo()
        elif opcion == 4:
            self.buscar_por_atributos()
        elif opcion == 5:
            self.mostrar_recorrido_por_niveles()
        elif opcion == 6:
            print("Saliendo...")
        else:
            print("Opción no válida. Intente nuevamente.")

    def insertar_nodo(self):
        """
        Solicita al usuario los datos de una nueva película y la inserta en el árbol AVL.
        """
        try:
            titulo = input("Escriba el Título de la película: ")
            anio = int(input("Ingrese Año de estreno de la película: "))
            ganancias_mundiales = float(input("Ingrese los ingresos a nivel mundial: "))
            ganancias_domesticas = float(input("Ingrese los ingresos a nivel nacional: "))
            ganancias_extranjeras = float(input("Ingrese los ingresos a nivel internacional: "))

            porcentaje_ganancias_domesticas = (ganancias_domesticas / ganancias_mundiales) * 100
            porcentaje_ganancias_extranjeras = (ganancias_extranjeras / ganancias_mundiales) * 100

            pelicula = Nodo(titulo, ganancias_mundiales, ganancias_domesticas, porcentaje_ganancias_domesticas,
                            ganancias_extranjeras, porcentaje_ganancias_extranjeras, anio)
            self.arbol_avl.insertar(pelicula)
            print("Nodo insertado con éxito.")
            self.visualizar_arbol("despues_de_insertar")
        except ValueError:
            print("Error: Ingrese valores válidos.")

    def eliminar_nodo(self):
        """
        Solicita al usuario el título de una película y la elimina del árbol AVL.
        """
        try:
            titulo = input("Escriba el Título de la película a eliminar: ")
            self.arbol_avl.eliminar(titulo)
            print("Nodo eliminado con éxito.")
            self.visualizar_arbol("despues_de_eliminar")
        except ValueError:
            print("Error: Ingrese valores válidos.")

    def buscar_nodo(self):
        """
        Busca una película por su título y muestra información detallada del nodo.
        """
        try:
            titulo = input("Escriba el Título de la película: ")
            nodo = self.arbol_avl.buscar(titulo)

            if nodo is None:
                print("Nodo no encontrado.")
            else:
                print(f"El nivel del nodo es: {self.arbol_avl.obtener_nivel_nodo(nodo.titulo)}")
                print(f"El factor de equilibrio es: {self.arbol_avl.factor_equilibrio(nodo)}")
                padre = self.arbol_avl.encontrar_padre(nodo.titulo)
                print(f"El padre del Nodo es: {padre.titulo if padre else 'Ninguno'}")
                abuelo = self.arbol_avl.encontrar_abuelo(nodo.titulo)
                print(f"El Abuelo del Nodo es: {abuelo.titulo if abuelo else 'Ninguno'}")
                tio = self.arbol_avl.encontrar_tio(nodo.titulo)
                print(f"El Tío del Nodo es: {tio.titulo if tio else 'Ninguno'}")
        except ValueError:
            print("Error: Ingrese valores válidos.")

    def buscar_por_atributos(self):
        """
        Busca películas que cumplan con criterios específicos de año y ganancias.
        """
        try:
            anio = int(input("Escriba el Año de las películas: "))
            valor = float(input("Digite el valor mínimo deseado para los ingresos internacionales: "))
            self.arbol_avl.buscar_por_atributos(anio, valor)
        except ValueError:
            print("Error: Ingrese valores válidos.")

    def mostrar_recorrido_por_niveles(self):
        """
        Muestra el recorrido por niveles del árbol AVL.
        """
        try:
            niveles = self.arbol_avl.recorrido_por_niveles()
            for i, nivel in enumerate(niveles, 1):
                print(f"Nivel {i}:")
                print(" | ".join(str(nodo) for nodo in nivel))
        except ValueError:
            print("Error: Ingrese valores válidos.")

    def visualizar_arbol(self, operacion):
        """
        Genera una visualización gráfica del árbol AVL actual.

        :param operacion: Descripción de la operación realizada (para nombrar el archivo).
        """
        self.contador_visualizaciones += 1
        nombre_archivo = f"ÁrbolAVL - {self.contador_visualizaciones}"
        self.arbol_avl.visualizar(nombre_archivo)
        print(f"Árbol AVL visualizado en el archivo '{nombre_archivo}.png'.")

if __name__ == "__main__":
    sistema = SistemaGestionPeliculasAVL()
    sistema.ejecutar()

Árbol AVL visualizado en el archivo 'ÁrbolAVL - 1.png'.
Árbol AVL de películas creado desde CSV y visualizado.

        Digite el número de la opción que desea aplicar:
        1. Insertar Nodo.
        2. Eliminar Nodo.
        3. Buscar Nodo a partir del nombre.
        4. Buscar los nodos que cumplan con los criterios establecidos.
        5. Mostrar el recorrido por niveles.
        6. Salir.
        
Opción seleccionada: 5
Nivel 1:
Título: Mission: Impossible II, Año: 2000
Nivel 2:
Título: Gladiator, Año: 2000 | Título: What Lies Beneath, Año: 2000
Nivel 3:
Título: Chicken Run, Año: 2000 | Título: How the Grinch Stole Christmas, Año: 2000 | Título: The Perfect Storm, Año: 2000 | Título: What Women Want, Año: 2000
Nivel 4:
Título: Charlie's Angels, Año: 2000 | Título: Dinosaur, Año: 2000 | Título: Gone in 60 Seconds, Año: 2000 | Título: Meet the Parents, Año: 2000 | Título: Scary Movie, Año: 2000 | Título: Unbreakable, Año: 2000 | Título: X0Men, Año: 2000
Nivel 5:
Título: Cast Away